In [13]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import os

from sklearn.svm import SVC
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score

In [5]:
# get working directory and remove last folder
# TODO: make this more robust
wd = os.path.dirname(os.getcwd())
os.chdir(wd)
print("Working Directory: ", os.getcwd())

# the working directory should be .../student-performance

Working Directory:  /Users/marvin/workspace/kaggle_gameplay/student-performance


In [15]:
df = pd.read_csv("data/processed/df_0_4_flattened.csv", index_col=0)
df.head()

,session_id_1,level_group_1,event_name_1,name_1,fqid_1,room_fqid_1,text_fqid_1,fullscreen_1,hq_1,music_1,...,room_coor_x_std_5,room_coor_y_std_5,screen_coor_x_std_5,screen_coor_y_std_5,hover_duration_std_5,difference_clicks_std_5,index_sum_of_actions_5,difference_clicks_max_5,elapsed_time_max_5,clicks_per_second_5
0,20090312431273200,0-4,6,3,5,1,6,0,0,1,...,422.014526,41.143421,181.938446,39.566540,164.755875,8.380692,13.0,30.837000,38.620998,0.336604
1,20090312433251036,0-4,5,3,4,1,4,0,0,0,...,352.261993,63.718025,181.674881,68.807640,-1.000000,10.578836,12.0,37.409000,47.641998,0.251879
2,20090312455206810,0-4,5,3,4,1,4,1,1,1,...,284.153931,34.723782,197.985825,54.664265,-1.000000,7.594211,14.0,28.743999,40.616001,0.344692
3,20090313091715820,0-4,5,3,4,1,4,1,1,1,...,337.755798,117.306465,187.151123,129.229507,-1.000000,12.563816,14.0,47.848999,60.853001,0.230063
4,20090313571836404,0-4,5,3,5,1,6,0,0,1,...,395.347290,57.395401,155.039871,56.676224,625.082397,9.021479,12.0,31.920000,41.750999,0.287418


In [7]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 23562 entries, 0 to 23561
Columns: 104 entries, session_id_1 to clicks_per_second_5
dtypes: float64(70), int64(33), object(1)
memory usage: 18.9+ MB


In [34]:
dt = df.dtypes.to_dict()
for d in dt:
    print(d, dt[d])

session_id_1 int64
level_group_1 object
event_name_1 int64
name_1 int64
fqid_1 int64
room_fqid_1 int64
text_fqid_1 int64
fullscreen_1 int64
hq_1 int64
music_1 int64
hover_duration_mean_1 float64
difference_clicks_mean_1 float64
elapsed_time_std_1 float64
page_std_1 float64
room_coor_x_std_1 float64
room_coor_y_std_1 float64
screen_coor_x_std_1 float64
screen_coor_y_std_1 float64
hover_duration_std_1 float64
difference_clicks_std_1 float64
index_sum_of_actions_1 float64
difference_clicks_max_1 float64
elapsed_time_max_1 float64
clicks_per_second_1 float64
session_id_2 int64
event_name_2 int64
name_2 int64
fqid_2 int64
room_fqid_2 int64
text_fqid_2 int64
hover_duration_mean_2 float64
difference_clicks_mean_2 float64
elapsed_time_std_2 float64
page_std_2 float64
room_coor_x_std_2 float64
room_coor_y_std_2 float64
screen_coor_x_std_2 float64
screen_coor_y_std_2 float64
hover_duration_std_2 float64
difference_clicks_std_2 float64
index_sum_of_actions_2 float64
difference_clicks_max_2 float6

In [63]:
X = df.iloc[:, 2:63:].values
print(type(X))
print(X.shape)
print(X)
X = X.astype(np.float64)
X.dtype

# 63 is making problems: too large for float64

<class 'numpy.ndarray'>
(23562, 61)
[[  6.           3.           5.         ...  39.           2.21499991
   34.25299835]
 [  5.           3.           4.         ...  32.           8.77999973
   91.22299957]
 [  5.           3.           4.         ...  42.           5.7670002
   32.01100159]
 ...
 [  5.           3.           5.         ...  34.           4.35900021
   46.97100067]
 [  7.           3.           6.         ...  45.           2.17000008
   27.78899956]
 [  7.           3.           6.         ...  51.           4.96999979
  105.43599701]]


dtype('float64')

In [47]:
labels = pd.read_csv("data/processed/labels.csv")
labels.head()


,session,correct,q
0,20090312431273200,1,1
1,20090312433251036,0,1
2,20090312455206810,1,1
3,20090313091715820,0,1
4,20090313571836404,1,1


In [48]:
print(labels.shape)
labels.info()

(424116, 3)
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 424116 entries, 0 to 424115
Data columns (total 3 columns):
 #   Column   Non-Null Count   Dtype
---  ------   --------------   -----
 0   session  424116 non-null  int64
 1   correct  424116 non-null  int64
 2   q        424116 non-null  int64
dtypes: int64(3)
memory usage: 9.7 MB


In [49]:
# y = labels[(labels['q'] >= 0) & (labels['q'] <= 3)]
y = labels[labels['q'] == 1]
y = y['correct'].values
print(y.shape)
print(y)

(23562,)
[1 0 1 ... 1 0 0]


In [57]:
# Creating training and test split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=1, stratify = y)
 
# Feature Scaling
 
sc = StandardScaler()
sc.fit(X_train)
X_train_std = sc.transform(X_train)
X_test_std = sc.transform(X_test)
 
# Training a SVM classifier using SVC class
svm = SVC(kernel= 'rbf', random_state=1, C=0.1)
svm.fit(X_train_std, y_train)
 
# Mode performance
 
y_pred = svm.predict(X_test_std)
print('Accuracy: %.3f' % accuracy_score(y_test, y_pred))

ValueError: Input X contains infinity or a value too large for dtype('float64').

In [58]:
for i in range(2,101):
    try:
        X = df.iloc[:, i:i+1].values
        # Creating training and test split
        X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=1, stratify = y)
        
        # Feature Scaling
        
        sc = StandardScaler()
        sc.fit(X_train)
        X_train_std = sc.transform(X_train)
        X_test_std = sc.transform(X_test)
    except: 
        print(f"This index {i} does not work.")
        

This index 63 does not work.


In [59]:
df.columns[63]

'clicks_per_second_3'

In [61]:
t = df.iloc[:, 63].values
t.dtype

dtype('float64')